# Download

データセットをZIPにかためてDownloadします。

In [ ]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob
import time
import datetime
import subprocess

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [ ]:
import subprocess
import os 
import shutil 
import datetime 

task_dropdown = ipywidgets.Dropdown(options=[], description='タスク')
dataset_dropdown = ipywidgets.Dropdown(options=[], description='データセット')
zip_compress_button = ipywidgets.Button(description='ZIPに圧縮')
xy_data_count_widget = ipywidgets.IntText(description='XYデータ数')
speed_data_count_widget = ipywidgets.IntText(description='速度データ数')
zip_file_dropdown = ipywidgets.Dropdown(options=[], description='ZIPファイル')
zip_file_update_button = ipywidgets.Button(description='ZIPリストの更新')
zip_decompress_button = ipywidgets.Button(description='ZIPを解凍')

base_path = "./"

def update_count(change):
    global base_path
    
    task = task_dropdown.value
    dataset = dataset_dropdown.value
    
    xy_path = base_path + "/" + task + "/" + dataset + "/xy/"
    speed_path = base_path + "/" + task + "/" + dataset + "/speed/"

    xy_is_dir = os.path.isdir(xy_path)
    speed_is_dir = os.path.isdir(speed_path)

    xy_file_count = 0
    speed_file_count = 0

    if xy_is_dir:
        xy_file_count = sum(os.path.isfile(os.path.join(xy_path,name)) for name in os.listdir(xy_path))
        xy_data_count_widget.value = xy_file_count
    else:
        xy_data_count_widget.value = 0
        
    if speed_is_dir:
        speed_file_count = sum(os.path.isfile(os.path.join(speed_path,name)) for name in os.listdir(speed_path))
        speed_data_count_widget.value = speed_file_count
    else:
        speed_data_count_widget.value = 0
        
dataset_dropdown.observe(update_count, names='value')
        
def change_dataset(change):
    global base_path

    if not change['new']:
        write_log("タスクが選択されていません。")
        return
    
    try:
        path = os.path.join(base_path, str(change['new']))
        write_log(f"change_dataset: {path}")
        if not os.path.exists(path):
            write_log(f"{path}が存在していません。")
            return
        
        dirs = [f for f in os.listdir(path) if os.path.isdir(os.path.join(path, f))
                and not f.startswith(".")
                and f not in {"__pycache__"}]
        dirs = sorted(dirs)
        dataset_dropdown.options = dirs
    except Exception as e:
        write_log(f"Error: {str(e)}")
        dataset_dropdown.options = []

def change_task():
    global base_path
    
    if not os.path.exists(base_path):
        write_log(f"{base_path}が存在していません。")
        return
    
    try:
        dirs = [f for f in os.listdir(base_path) 
                if os.path.isdir(os.path.join(base_path, f))
                and not f.startswith(".")
                and f not in {"__pycache__", "model_c", "model", "model_trt", "video", "zip"}]
        dirs = sorted(dirs)
        task_dropdown.options = dirs
    except Exception as e:
        write_log(f"Error: {str(e)}")
        task_dropdown.options = []

task_dropdown.observe(change_dataset, names='value')
change_task()

def get_zip_list(change):
    global base_path
    try:
        zip_upload_path = os.path.join(base_path, "zip/upload/")
        zip_files = [f for f in os.listdir(zip_upload_path) 
                     if os.path.isfile(os.path.join(zip_upload_path, f)) and f.lower().endswith('.zip')]
        zip_files = sorted(zip_files)
        zip_file_dropdown.options = zip_files
    except Exception as e:
        write_log(f"Error: {str(e)}")
        zip_file_dropdown.options = []

get_zip_list("")
zip_file_update_button.on_click(get_zip_list)

In [ ]:
import zipfile

def compress_zip(change):
    global base_path
    
    start_time = datetime.datetime.now()
    write_log(f"処理開始: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

    try:
        zip_download_path = os.path.join(base_path, "zip/download/")
        if not os.path.exists(zip_download_path):
            os.makedirs(zip_download_path)
            write_log("zipフォルダを生成")

        zip_filename = f"{task_dropdown.value}_{dataset_dropdown.value}.zip"
        zip_filepath = os.path.join(zip_download_path, zip_filename)

        # 対象のディレクトリ
        target_dir = os.path.join(base_path, f"{task_dropdown.value}/{dataset_dropdown.value}")

        
        # ZIPファイルを作成
        with zipfile.ZipFile(zip_filepath, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(target_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    # アーカイブ内の相対パスを保持
                    arcname = os.path.relpath(file_path, start=base_path)
                    zipf.write(file_path, arcname)  # 直接ファイルを書き込む
        write_log(f"保存ファイル名: {zip_filepath}")

    except Exception as e:
        write_log(f"Error: {str(e)}")

    end_time = datetime.datetime.now()
    elapsed_time = (end_time - start_time).total_seconds()
    write_log(f"処理時間: {elapsed_time:.2f} 秒")
    
#zip_compress_button.on_click(compress_zip)

In [ ]:
import subprocess
import datetime

def compress_zip_with_command(change):
    global base_path
    
    start_time = datetime.datetime.now()
    write_log(f"処理開始: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

    try:
        zip_download_path = os.path.join(base_path, "zip/download/")
        if not os.path.exists(zip_download_path):
            os.makedirs(zip_download_path)
            write_log("zipフォルダを生成")

        destination_foldername = f"{task_dropdown.value}_{dataset_dropdown.value}"
        compress_foldername = f"{task_dropdown.value}/{dataset_dropdown.value}"
        zip_filename = f"{destination_foldername}.zip"
        destination_path = os.path.join(base_path, compress_foldername)

        # 圧縮コマンドの実行
        zip_command = ["zip", "-ro", os.path.join(zip_download_path, zip_filename), destination_path]
        subprocess.run(zip_command, check=True)
        write_log(f"保存ファイル名: {os.path.join(zip_download_path, zip_filename)}")

    except Exception as e:
        write_log(f"Error: {str(e)}")

    end_time = datetime.datetime.now()
    elapsed_time = (end_time - start_time).total_seconds()
    write_log(f"処理時間: {elapsed_time:.2f} 秒")
    write_log(f"./zip/download/フォルダから {zip_filename} をダウンロードしてください。")
    
zip_compress_button.on_click(compress_zip_with_command)

In [ ]:
zip_folder_path = os.path.join(base_path, "zip/upload/")
if not os.path.exists(zip_folder_path):
    os.makedirs(zip_folder_path)
        
def decompress_zip_with_command(change):
    start_time = datetime.datetime.now()
    write_log(f"処理開始: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

    try:
        base_path = "./"
        zip_folder_path = os.path.join(base_path, "zip/upload/")
        zip_filename = zip_file_dropdown.value  
        zip_filepath = os.path.join(zip_folder_path, zip_filename)

        destination_dir = base_path
        if not os.path.exists(destination_dir):
            os.makedirs(destination_dir)

        # Decompress using unzip command with extra options
        unzip_command = ["unzip", "-o", zip_filepath, "-d", destination_dir]
        result = subprocess.run(unzip_command, check=False, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        
        if result.returncode != 0:
            raise subprocess.CalledProcessError(result.returncode, unzip_command, output=result.stdout, stderr=result.stderr)
        
        write_log(f"解凍ファイル名: {zip_filepath}")
        write_log(f"解凍先ディレクトリ: {destination_dir}")

    except subprocess.CalledProcessError as e:
        write_log(f"subprocess error: {e} - {e.stderr.decode().strip()}")
    except Exception as e:
        write_log(f"Error: {str(e)}")

    end_time = datetime.datetime.now()
    elapsed_time = (end_time - start_time).total_seconds()
    write_log(f"処理時間: {elapsed_time:.2f} 秒")

zip_decompress_button.on_click(decompress_zip_with_command)

In [ ]:
separator = ipywidgets.HTML('<hr style="border-color:gray;margin:10px 0"/>')
title1 = ipywidgets.HTML('<b>【Donwload　ZIP】</b> データセットを選択し、ZIPに圧縮します。zip/downloadフォルダにファイルは生成されます。')
title２ = ipywidgets.HTML('<b>【Upload　ZIP】</b> zip/uploadフォルダにZIPをアップロードしてください。解凍します。')

zip_widget = ipywidgets.VBox([
    separator,
    title1,
    ipywidgets.HBox([dataset_dropdown,task_dropdown]),
    ipywidgets.HBox([xy_data_count_widget,speed_data_count_widget]),
    zip_compress_button,
    process_widget,
    separator,
    title2,
    ipywidgets.HBox([zip_file_dropdown,zip_file_update_button,zip_decompress_button]),
    process_widget
])
display(zip_widget)